In [1]:
# 11/21(금) 15:30

# Convolutional Neural Network 모델 정의

In [1]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [2]:
# Conv2d 생성
## 입력 shape: (Batch, Channel, Height, Width)  (100, 10, 100, 200)
layer = nn.Conv2d(
    in_channels=10,   # 입력 channel 수
    out_channels=5,  # Filter (kernel)의 개수 = Feature map의 개수
    kernel_size=3,   # Filter의 크기 (3 * 3) Height와 Width의 크기가 같으면 정수. 
    stride=1,        # 이동 보폭 (Default = 1)
    padding=1      
)

In [3]:
input_data = torch.ones(1, 10, 10, 10)
output = layer(input_data)
output.shape

torch.Size([1, 5, 10, 10])

In [5]:
input_data = torch.ones(1, 10, 10, 10)
output = layer(input_data)
output.shape

torch.Size([1, 5, 10, 10])

In [6]:
layer.weight.shape  # 결과: torch.Size([5, 10, 3, 3]). 
# [Filter 개수: 5, 입력 채널 수: 10, filter-height:3, filter-width:3]
# (out_channels, in_channels, kernel_size, kernel_size)

torch.Size([5, 10, 3, 3])

In [7]:
layer.bias.shape  # 결과: torch.Size([5]). filter 별로 한 개씩.

torch.Size([5])

In [19]:
pool_layer = nn.MaxPool2d(
    kernel_size=2,  # pool 영역 크기 (2 * 2)
    stride=2,       # 이동 크기. kernel_size와 동일.
    padding=0
)

In [9]:
print(input_data.shape)
pool_output = pool_layer(input_data)
print(pool_output.shape)

torch.Size([1, 10, 10, 10])
torch.Size([1, 10, 5, 5])


In [20]:
input_data = torch.randn(1, 1, 5, 5)

print(input_data.shape)
pool_output = pool_layer(input_data)
print(pool_output.shape)

torch.Size([1, 1, 5, 5])
torch.Size([1, 1, 2, 2])


In [13]:
input_data

tensor([[[[-0.8589,  1.4193, -0.9620,  0.5543, -1.2227],
          [-0.8978,  0.1540, -2.2233,  0.6342, -1.1793],
          [-0.6150, -0.1232, -0.9120,  1.5637,  0.1271],
          [ 1.3266,  0.3798,  0.4952, -0.4290,  0.0162],
          [ 0.7466,  0.9015,  1.2378,  1.2700, -0.8286]]]])

In [11]:
pool_output

tensor([[[[1.9252, 1.9673],
          [1.3628, 1.7152]]]])

# MNIST

In [14]:
import os

import torch
from torch import nn
import torchinfo

import matplotlib.pyplot as plt
import numpy as np

from module.data import load_mnist_dataset, load_fashion_mnist_dataset
from module.train import fit

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [15]:
# 하이퍼파라미터 지정
EPOCH = 10
BATCH_SIZE = 256
LEARNING_RATE = 0.001
DATA_ROOT_DIR = "datasets"

## Data 준비

In [16]:
train_loader = load_mnist_dataset(DATA_ROOT_DIR, BATCH_SIZE, True)
test_loader = load_mnist_dataset(DATA_ROOT_DIR, BATCH_SIZE, False)

## CNN 모델 정의

In [28]:
# CNN - layer block
# ConvolutionLayer -> Activation -> Pooling Layer
# ConvolutionLayer -> Activation -> ConvolutionLayer -> Activation-> Pooling Layer

# ConvolutionLayer -> BatchNormalization -> Activation -> Dropout -> Pooling Layer

## 구조: filter 개수는 늘려주고(channel-depth) feature map의 size는 줄이는 방식으로 구성.
# depth: Conv2d,  size: MaxPool2d

class CNNModel(nn.Module):

    def __init__(self, dropout_rate=0.2):
        
        super().__init__()
        

        self.b1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1, 
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding="same"                                  
            ),
            nn.BatchNorm2d(32),  # 위의 out_channel과 같은 값
            nn.ReLU(),
            # nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(
                kernel_size=2, 
                stride=2
            )
        )

        self.b2 = nn.Sequential(
            nn.Conv2d(
                in_channels=32, 
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding="same"
            ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.b3 = nn.Sequential(
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding="same"
            ),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )
        # 추론기(분류기) - Fully Conntected Layer(nn.Liear)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=4*4*128, out_features=10)
        )

    def forward(self, X):
        output = self.b1(X)
        output = self.b2(output)
        output = self.b3(output)
        output = self.classifier(output)
        return output

In [29]:
torchinfo.summary(CNNModel(dropout_rate=0.5), (1, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
CNNModel                                 [1, 10]                   --
├─Sequential: 1-1                        [1, 32, 14, 14]           --
│    └─Conv2d: 2-1                       [1, 32, 28, 28]           320
│    └─BatchNorm2d: 2-2                  [1, 32, 28, 28]           64
│    └─ReLU: 2-3                         [1, 32, 28, 28]           --
│    └─MaxPool2d: 2-4                    [1, 32, 14, 14]           --
├─Sequential: 1-2                        [1, 64, 7, 7]             --
│    └─Conv2d: 2-5                       [1, 64, 14, 14]           18,496
│    └─BatchNorm2d: 2-6                  [1, 64, 14, 14]           128
│    └─ReLU: 2-7                         [1, 64, 14, 14]           --
│    └─MaxPool2d: 2-8                    [1, 64, 7, 7]             --
├─Sequential: 1-3                        [1, 128, 4, 4]            --
│    └─Conv2d: 2-9                       [1, 128, 7, 7]            73,856
│    

## Train

In [30]:
# 모델
model = CNNModel(dropout_rate=0.5).to(device)

# loss 함수
loss_fn = nn.CrossEntropyLoss() 

# 옵티마이저
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [31]:
os.makedirs("saved_models", exist_ok=True)

save_path = "saved_models/mnist_cnn_model.pt"
result = fit(
    train_loader, test_loader, model, loss_fn, optimizer, EPOCH,
    save_model_path=save_path,
    device=device, 
    mode="multi"
)

Epoch[1/10] - Train loss: 0.04258 Train Accucracy: 0.98555 || Validation Loss: 0.04032 Validation Accuracy: 0.98740
저장: 1 - 이전 : inf, 현재: 0.04031606172211468


KeyboardInterrupt: 

In [ ]:
model = torch.load(save_path, weights_only=False)

In [ ]:
# 최종 평가
from module.train import test_multi_classification
loss, acc = test_multi_classification(
    test_loader, model, loss_fn, device
)
loss, acc

In [ ]:
## 정성적 평가 - 실제 image 파일로 확인
from PIL import Image
img = Image.open("test_img/num/eight.png")
type(img)

In [ ]:
from torchvision import transforms
from PIL import Image

@torch.no_grad
def predict(path, model):
    img = Image.open(path)

    # color -> grayscale
    img = img.convert('L') # 'L': grayscale, "RGB": color

    # resize
    input_tensor = transforms.Resize((28, 28))(img)

    # PIL.Image -> torch.Tensor  변환, 정규화 (0 ~ 1)
    input_tensor = transforms.ToTensor()(input_tensor)

    # batch 축(dummy 축)을 추가.
    input_tensor = input_tensor.unsqueeze(dim=0)

    # device로 이동
    input_tensor = input_tensor.to(device)

    model.eval()
    model = model.to(device)
    
    result = model(input_tensor)
    sm = nn.Softmax(dim=-1)
    result_proba = sm(result)
    final_result = result_proba.max(dim=-1)
    return {"class":final_result.indices[0], 
            "확률":final_result.values[0]}

In [ ]:
from glob import glob

img_path_list = glob("test_img/num/*.png")
for img_path in img_path_list:
    result = predict(img_path, model)
    print(f"{img_path}, 추론class: {result['class']}, 확률: {result['확률']}")